In [1]:
## Basic import and DF creation
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML

# Specify the path to your Excel file
file_path = r"09_Outputs\main_data.xlsx"

# Load the Excel file into a DataFrame using the openpyxl engine
df_main = pd.read_excel(file_path, engine='openpyxl')

# Remove quotation marks from the 'Entity' column
df_main['Entity'] = df_main['Entity'].str.replace('"', '')

# Retrieve unique countries from the DataFrame and store them as a list of strings
all_countries = df_main['Entity'].drop_duplicates().astype(str).tolist()

# Display the first 5 rows of the DataFrame for a quick overview
df_main.head()


# Assuming df_main is already available
filtered_df = df_main

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']
left_container_width = '270px'

# Custom sorting function
def custom_sort(country):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    if country in selected_countries:
        return (-1, selected_countries.index(country))  # Selected countries come first
    else:
        return (1, country)  # All other countries come last

# Update list of countries based on user search input
def update_country_list(*args):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    search_input = search_box.value
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    updated_countries = list(set(search_results + selected_countries))
    
    updated_countries = sorted(updated_countries, key=custom_sort)
    
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    checkboxes_container.children = tuple(updated_checkboxes)
    
    for cb in checkboxes_container.children:
        cb.observe(plot_selected_column, names='value')
        cb.observe(update_country_list, names='value') 

# The plot_selected_column function
def plot_selected_column(change=None):
    column = column_dropdown.value
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    with out:
        clear_output(wait=True)
        fig = go.Figure()
        for country in selected_countries:
            country_data = filtered_df[filtered_df['Entity'] == country]
            fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country, showlegend=True))
        
        # Added dotted grid layout
        fig.update_layout(title=column, 
                          xaxis=dict(title='Year', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          yaxis=dict(title=column, showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          plot_bgcolor='white', paper_bgcolor='white')
        fig.show()


checkboxes_container = widgets.VBox([], layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country, layout=widgets.Layout(width="100%", padding="0px")) for country in sorted(all_countries, key=custom_sort)]

for cb in country_checkboxes:
    cb.observe(plot_selected_column, names='value')

checkboxes_container.children = tuple(country_checkboxes)

# Creating the search box widget
search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
search_box.observe(update_country_list, names='value')

clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
clear_button.on_click(lambda change: setattr(search_box, 'value', ''))

out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))

columns_to_plot = filtered_df.columns[filtered_df.columns.get_loc('Code')+1:]
column_dropdown = widgets.Dropdown(options=columns_to_plot, description='Select Column:')
column_dropdown.observe(plot_selected_column, names='value')

left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container], layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))

display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
</style>
"""))

display(column_dropdown)
display(widgets.HBox([left_container, out]))
plot_selected_column()
# Initial setup
update_country_list()

Dropdown(description='Select Column:', options=('Annual CO₂ emissions', 'Renewables (TWh growth - equivalent)'…